In [1]:
import numpy as np

In [3]:
vocabulary = np.load('./tokens/vocab.npy')
X_train = np.load('./tokens/X_train.npy')
X_test = np.load('./tokens/X_test.npy')
y_train = np.load('./tokens/y_train.npy')
y_test = np.load('./tokens/y_test.npy')
X_train = X_train.astype(np.int32)
X_test = X_test.astype(np.int32)
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

In [5]:
# Paso 1: Importar las librerías necesarias
import tensorflow as tf
# Paso 3: Cargar el modelo .h5 (ajusta la ruta si es necesario)
# Por ejemplo: '/content/drive/MyDrive/modelo.h5'
ruta_modelo = './lstm_v66.h5'
model = tf.keras.models.load_model(ruta_modelo)

In [6]:
predictions = model(X_test).numpy()

In [7]:
class_indices = predictions.argmax(axis=1)

In [8]:
X_test = np.array(X_test, dtype=np.float32)
class_indices = np.array(class_indices, dtype=np.int64)
baselines = np.zeros_like(X_test, dtype=np.float32)

In [14]:
print(baselines)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [9]:
print("X_test:", type(X_test), X_test.shape, X_test.dtype)
print("baselines:", type(baselines), baselines.shape, baselines.dtype)
print("class_indices:", type(class_indices), class_indices.shape, class_indices.dtype)

X_test: <class 'numpy.ndarray'> (76, 730) float32
baselines: <class 'numpy.ndarray'> (76, 730) float32
class_indices: <class 'numpy.ndarray'> (76,) int64


In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 730, 200)          200000    
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               135680    
 onal)                                                           
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 344001 (1.31 MB)
Trainable params: 344001 (1.31 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
from alibi.explainers import IntegratedGradients

/home/pajaro/anaconda3/envs/ali_env_v1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-12 15:10:56,881	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [13]:
n_steps = 50
method = "gausslegendre"
internal_batch_size = 100
ig  = IntegratedGradients(model,
                          layer=None,
                          n_steps=n_steps,
                          method=method,
                          internal_batch_size=internal_batch_size)
explanation = ig.explain(X_test,
                         baselines=baselines,
                         target=class_indices,
                         attribute_to_layer_inputs=False)

ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.